In [1]:
import numpy as np
import random

# Value Function
3 dimensional array, where first index is \[cardsum(12 to 21) - 12\] (_for indexing_); second index is 0 if no usable ace and 1 if any; third index is dealer card \[(1 to 10)-1\](_for indexing_).

In [2]:
V = np.array([[[0.]*10]*2]*10)
visit_count = np.array([[[0.]*10]*2]*10)

In [3]:
visit_count

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.,

# BlackJack Monte-Carlo Strategy

In [4]:
def pi_20(hand, dealer_card):
    player_sum = sum(hand)
    
    if 1 in hand and player_sum < 12:
            player_sum += 10
    
    if player_sum < 20:
        return 'hit'
    else:
        return 'stick'

In [5]:
def update_values(states, r, V, visit_count):
    for h, a, d in states:
        V[h][a][d] = V[h][a][d] + (1/visit_count[h][a][d])*(r - V[h][a][d]) 

In [6]:
array =([(3.1, 2), (2.1, 4)])

# One Hand Blackjack Simulator

In [7]:
def blackjack_hand_result(V, visit_count):
    #Deck of cards, 4 types of 10
    
    deck=np.array([1,2,3,4,5,6,7,8,9,10,10,10,10])
    
    #Creating a random hand if none is input
    
    player_hand=[random.choice(deck), random.choice(deck)]
        
    dealer_card=random.choice(deck)
    
    # keeps track of all states that the hand has been in during one play
    states = []
    
    player_sum = sum(player_hand)
    
    # an ace that can be used as an 11 i.e. the sum of the cards is less than 12
    usable_ace = 0
    if 1 in player_hand:
        usable_ace = 1
        player_sum += 10
    
    
    if player_sum > 11:
        states.append((player_sum-12, usable_ace, dealer_card-1))
        visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
        
    # keeps track of dealers hand
    dealer_cards=[dealer_card]
    

    # Seeing if player hit blackjack
    if player_sum == 21:

        #Seeing if casino also hit blackjack, in which case tie

        dealer_cards.append(random.choice(deck))
        if 1 in (dealer_cards) and sum(dealer_cards)==11:
            update_values(states, 0, V, visit_count)
            return 0
        
        update_values(states, 1, V, visit_count)
        return 1
    

    else:
    # Seeing how often it says to 'hit'

        while pi_20(player_hand, dealer_card) == 'Hit':
            #Adding one card for every hit
            new_card = random.choice(deck)
            player_hand.append(new_card)
            
            player_sum += new_card
            
            if new_card == 1 and player_sum < 12:
                usable_ace = 1
                player_sum += 10
            
            if usable_ace == 1 and player_sum > 21:
                player_sum -= 10
                usable_ace == 0

            #Player loses bet if hand goes above 21
            if player_sum>21:
                update_values(states, -1, V, visit_count)
                return -1
            
            if player_sum > 11:
                states.append((player_sum-12, usable_ace, dealer_card-1))
                visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
            
    while True:
        #Plays out the blackjack hand from dealer's side

        #Give dealer extra card if loop hasn't broken
        dealer_cards.append(random.choice(deck))


        #Keep track of sum of dealer's cards
        dealer_score= sum(dealer_cards)

        #Keep track of soft score if dealer has an ace

        soft_score= dealer_score
        if dealer_score<=11 and 1 in dealer_cards:
            soft_score+=10

        #If dealer gets blackjack you lose even if you have 21
        if len(dealer_cards)==2 and soft_score==21:
            return -1
        
        #Dealer stays on all 17s
        if soft_score>=17:

            #If dealer bust, player wins bet
            if soft_score>21:
                update_values(states, 1, V, visit_count)
                return 1

            #If player has more than dealer, player wins bet
            if player_sum>soft_score:
                update_values(states, 1, V, visit_count)
                return 1

            #Tie means no money changes hands
            if player_sum==soft_score:
                update_values(states, 0, V, visit_count)
                return 0

            #If player has lower, player loses bet
            if player_sum<soft_score:
                update_values(states, -1, V, visit_count)
                return -1


# Multiple Hand Sim, Test to see house edge in Blackjack

In [8]:
def blackjack_sim(n_hands, V, visit_count):
    for i in range(n_hands):
        blackjack_hand_result(V, visit_count)
     

In [9]:
blackjack_sim(50_000, V, visit_count)
V

array([[[-5.78782779e-01, -3.59331476e-01, -2.80653951e-01,
         -1.62393162e-01, -1.45510836e-01, -2.66272189e-01,
         -5.05681818e-01, -5.24096386e-01, -5.63157895e-01,
         -5.76387928e-01],
        [-7.75282588e-01, -4.66666667e-01, -5.00000000e-01,
         -2.00000000e-01, -7.69230769e-02, -4.00000000e-02,
         -4.00000000e-02, -5.38461538e-01, -6.00000000e-01,
         -4.83036204e-01]],

       [[-6.63274407e-01, -3.33333333e-01, -2.68456376e-01,
         -2.56578947e-01, -2.58064516e-01, -2.34421365e-01,
         -4.13897281e-01, -5.66197183e-01, -6.15894040e-01,
         -5.48383820e-01],
        [-6.97334828e-01, -4.28571429e-01, -2.91666667e-01,
         -3.33333333e-01, -1.81818182e-02,  2.12765957e-02,
         -3.50000000e-01, -5.17241379e-01, -4.04255319e-01,
         -5.94792159e-01]],

       [[-6.83638634e-01, -2.45161290e-01, -2.67123288e-01,
         -1.92982456e-01, -1.84466019e-01, -1.71314741e-01,
         -5.71906355e-01, -4.71830986e-01, -5.73

In [10]:
#Simulates a million hands to estimate house edge
house_edge=-100*(blackjack_sim(n_hands=1_000_000, bet=1) / 1_000_000)
house_edge #percentage

TypeError: blackjack_sim() got an unexpected keyword argument 'bet'

# Strategy Test: Double 7 vs 6

In [ ]:
#Player profit from following basic strategy
blackjack_sim(n_hands=100_000, player_h=[4,3], dealer_c=6, bet=1)


In [ ]:
blackjack_sim(n_hands=100_000, player_h=[4,3], dealer_c=6, bet=1, hard_c='Double')


# Strategy Test: Split 10s vs 6

In [ ]:
blackjack_sim(n_hands=100_000, player_h=[10,10], dealer_c=6, bet=1)


In [ ]:
blackjack_sim(n_hands=100_000, player_h=[10,10], dealer_c=6, bet=1, hard_c='Split')
